In [24]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_url(position,location):
    position = position.replace(' ','%20')
    location = location.replace(' ','%20') #Replacing white spaces with -, to generate proper url
    template = 'https://malaysia.indeed.com/jobs?q={}&l={}'
    #Applying string formatting
    url = template.format(position,location)#First {} is position, 2nd {} is location
    return url

#Getting job info from the card

def get_jobs(card):
    job_name = card.find('h2','jobTitle').text
    #anchor_tag = card.a
    #job_url = 'https://www.jobstreet.com.my'+anchor_tag['href']
    company_name = card.find('span','companyName').text
    job_location = card.find('div','companyLocation').text
    # Fetching job summary
    try:
        summary = ''
        tag_text = ''
        job_summary = card.find('div','heading6 tapItem-gutter result-footer')
        for eachLI in job_summary('li'):
            tag_text = eachLI.text
            summary = summary + tag_text + ','
        if ',' in summary:
            summary = summary.rstrip(',')
        else:
            summary = 'Not Available'
    except AttributeError:
        summary = 'Not Available'
        
    #Fetching job salary
    job_salary = card.find_all('div','salary-snippet')
    try:
        if len(job_salary) > 0 :
            job_salary = job_salary[0]
            salary = job_salary.text
        else:
            salary = 'Not Available'
    except IndexError:
        salary = 'Not Available'
    #Fetching job post date
    
    post_date = card.find('span','date').text 
    today = datetime.today().strftime('%Y-%m-%d')
    
    
    job_info = (job_name,company_name,job_location,summary,
               salary,post_date,today)
    
    return job_info

# Creating a main function

def main(position,location):
    records = []
    url = get_url(position,location)
    
    response = requests.get(url)
    
    #Creating BeautifulSoup object
    soup = BeautifulSoup(response.text,'html.parser')
    
    cards = soup.find_all('div','job_seen_beacon')
    
    for everyCard in cards:
        jobDetails = get_jobs(everyCard)
        records.append(jobDetails)
        
    #Using pandas dataframe to save all job info in csv
    
    col = ['Job_Name','Company_Name','Job_Location',
          'Job_Summary','Job_Salary','Post_Date','Today_Date']
    
    job_street_data = pd.DataFrame(records,columns=col)
    
    job_street_data.to_csv('C:\\Users\\user\\Desktop\\web scraping task\\IndeedData.csv')

In [25]:
main('java','selangor')

<h3><center>Made with Love by Fikri</center></h3>